In [7]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import pandas as pd

In [10]:
yt=pd.read_csv(r'C:\Users\Varishu Pant\Desktop\Praxis docs\Excel\YtDataOld.csv')  #Reading Data

In [5]:
def pipeline(data):
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    import random as rand
    import pickle as pkl
    import warnings
    warnings.filterwarnings('ignore')
    to_remove=['Unnamed: 0','Unnamed: 0.1','twitter_url','defaultAudioLanguage']  #Removing Unncessary Columns
    data=data.drop(to_remove,axis=1)
   

    weekdays=['Friday','Monday','Saturday','Sunday','Thursday','Tuesday','Wednesday']
    categories=['Film & Animation','Autos & Vehicles','Music','Pets & Animals','Sports','Travel & Events','Gaming','People & Blogs','Comedy','Entertainment','News & Politics','Howto & Style','Education','Science & Technology','Nonprofits & Activism']

    for i in range(0,7):
        data.day_uploaded[data.day_uploaded==i]=weekdays[i]
    for i in range(0,15):
        data.categoryId[data.categoryId==i]=categories[i]

    data['YearPublished']=0
    years=[2010,2011,2012,2013,2014,2015,2016]
    for i in range(0,7):
        data['YearPublished'][data[(data['months_old']>12*i) & (data['months_old']<=12*(i+1))].index]=years[i]
    
    data=data.drop(index=data[data['YearPublished']==0].index,axis=0)
    
    data=data.drop(index=data[data['duration']==0].index,axis=0)
    return(data)

In [29]:
yt_processed=pipeline(yt)    #Using complete dataset for recommendations

In [12]:
menu_day = widgets.Dropdown(
       options=['Not Decided','Monday', 'Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday'],
       value='Not Decided',
       description='Upload Day:')
menu_categories= widgets.Dropdown(
       options=['Not Decided','Film & Animation','Autos & Vehicles','Music','Pets & Animals','Sports','Travel & Events','Gaming','People & Blogs','Comedy','Entertainment','News & Politics','Howto & Style','Education','Science & Technology','Nonprofits & Activism'],
       value='Not Decided',
       description='Category:')

In [30]:
def predict(day,category,duration=None):
    data=yt_processed
    m_v_d_c=data[['viewCount','categoryId','day_uploaded']].groupby(['day_uploaded','categoryId'],as_index=False).mean()
    m_v_c=data[['viewCount','categoryId']].groupby(['categoryId'],as_index=False).mean()
    m_v_d=data[['viewCount','day_uploaded']].groupby(['day_uploaded'],as_index=False).mean()
    categories=['Film & Animation','Autos & Vehicles','Music','Pets & Animals','Sports','Travel & Events','Gaming','People & Blogs','Comedy','Entertainment','News & Politics','Howto & Style','Education','Science & Technology','Nonprofits & Activism']
    days=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    if day=='Not Decided':                                            
        if category=='Not Decided':
            v=data['viewCount'].mean()
            print('Expected Views:',round(v))
            print('\nRecommendation:For maximum views,upload on a Friday')
            return()
            
        else:
            v=int(m_v_c[m_v_c['categoryId']==category]['viewCount'])
            print('Expected Views:',round(v))
            data=m_v_d_c[m_v_d_c['categoryId']==category]
            
    else:
        if category=='Not Decided':
            v=int(m_v_d[m_v_d['day_uploaded']==day]['viewCount'])
            print('Expected Views:',round(v))
            data=m_v_d_c[m_v_d_c['day_uploaded']==day]
            
           
            
        else:
            v=int(m_v_d_c[(m_v_d_c['day_uploaded']==day) & (m_v_d_c['categoryId']==category)]['viewCount'])
            print('Expected Views:',round(v))
            return()
            
    
    s=str(data[data['viewCount']==data['viewCount'].max()]['categoryId'])
    cat=s[s.find(' ')+4:s.find('\n')]
    s=str(data[data['viewCount']==data['viewCount'].max()]['day_uploaded'])
    weekday=s[s.find(' ')+4:s.find('\n')]
                    
    print('\nRecommendation:For maximum views,upload a',cat,'video on a',weekday)
    return()


In [31]:
box = widgets.VBox([menu_day,menu_categories ])
button = widgets.Button(description='Execute')
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          predict(day=menu_day.value,category=menu_categories.value)
            # linking button and function together using a button's method
button.on_click(on_button_clicked)

In [32]:
box

In [33]:
widgets.VBox([button,out])